# Machine Learning Foundation

## Course 5, Part g: RNN DEMO

## Using RNNs to classify sentiment on IMDB data
For this exercise, we will train a "vanilla" RNN to predict the sentiment on IMDB reviews.  Our data consists of 25000 training sequences and 25000 test sequences.  The outcome is binary (positive/negative) and both outcomes are equally represented in both the training and the test set.

Keras provides a convenient interface to load the data and immediately encode the words into integers (based on the most common words).  This will save us a lot of the drudgery that is usually involved when working with raw text.

We will walk through the preparation of the data and the building of an RNN model.  Then it will be your turn to build your own models (and prepare the data how you see fit).

In [1]:
%load_ext tensorboard

In [2]:
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.datasets import imdb
from keras import initializers

2022-08-29 09:37:35.573885: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-29 09:37:35.573899: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 30  # maximum length of a sequence - truncate after this
batch_size = 32

In [4]:
## Load in the data.  The function automatically tokenizes the text into distinct integers
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

25000 train sequences
25000 test sequences


In [5]:
# This pads (or truncates) the sequences so that they are of the maximum length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 30)
x_test shape: (25000, 30)


In [6]:
x_train[123,:]  #Here's what an example sequence looks like

array([  219,   141,    35,   221,   956,    54,    13,    16,    11,
        2714,    61,   322,   423,    12,    38,    76,    59,  1803,
          72,     8, 10508,    23,     5,   967,    12,    38,    85,
          62,   358,    99], dtype=int32)

## Keras layers for (Vanilla) RNNs

In this exercise, we will not use pre-trained word vectors.  Rather we will learn an embedding as part of the Neural Network.  This is represented by the Embedding Layer below.

### Embedding Layer
`keras.layers.embeddings.Embedding(input_dim, output_dim, embeddings_initializer='uniform', embeddings_regularizer=None, activity_regularizer=None, embeddings_constraint=None, mask_zero=False, input_length=None)`

- This layer maps each integer into a distinct (dense) word vector of length `output_dim`.
- Can think of this as learning a word vector embedding "on the fly" rather than using an existing mapping (like GloVe)
- The `input_dim` should be the size of the vocabulary.
- The `input_length` specifies the length of the sequences that the network expects.

### SimpleRNN Layer
`keras.layers.recurrent.SimpleRNN(units, activation='tanh', use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None, activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, dropout=0.0, recurrent_dropout=0.0)`

- This is the basic RNN, where the output is also fed back as the "hidden state" to the next iteration.
- The parameter `units` gives the dimensionality of the output (and therefore the hidden state).  Note that typically there will be another layer after the RNN mapping the (RNN) output to the network output.  So we should think of this value as the desired dimensionality of the hidden state and not necessarily the desired output of the network.
- Recall that there are two sets of weights, one for the "recurrent" phase and the other for the "kernel" phase.  These can be configured separately in terms of their initialization, regularization, etc.






In [7]:
## Let's build a RNN

rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence and embeds it in a 50-dimensional vector
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

2022-08-29 09:37:39.794665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-29 09:37:39.794683: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-29 09:37:39.794697: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (damminhtien-OptiPlex-3070): /proc/driver/nvidia/version does not exist
2022-08-29 09:37:39.794979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
import datetime

In [9]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
## Note that most of the parameters come from the embedding layer
model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          1000000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 5)                 280       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 1,000,286
Trainable params: 1,000,286
Non-trainable params: 0
_________________________________________________________________


In [11]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/home/damminhtien/anaconda3/envs/base-ml/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

Epoch 1/10
782/782 [==============================] - 7s 8ms/step - loss: 0.6601 - accuracy: 0.6175 - val_loss: 0.6014 - val_accuracy: 0.6854
Epoch 2/10
 21/782 [..............................] - ETA: 3s - loss: 0.5798 - accuracy: 0.7262

2022-08-29 09:37:46.998878: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 240000000 exceeds 10% of free system memory.


782/782 [==============================] - 6s 7ms/step - loss: 0.5532 - accuracy: 0.7210 - val_loss: 0.5416 - val_accuracy: 0.7204
Epoch 3/10
  9/782 [..............................] - ETA: 5s - loss: 0.5396 - accuracy: 0.7222

2022-08-29 09:37:52.505963: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 240000000 exceeds 10% of free system memory.


782/782 [==============================] - 7s 8ms/step - loss: 0.4935 - accuracy: 0.7617 - val_loss: 0.5113 - val_accuracy: 0.7441
Epoch 4/10
 19/782 [..............................] - ETA: 4s - loss: 0.4564 - accuracy: 0.7845

...

782/782 [==============================] - 6s 7ms/step - loss: 0.4539 - accuracy: 0.7887 - val_loss: 0.4876 - val_accuracy: 0.7598
Epoch 5/10
 19/782 [..............................] - ETA: 4s - loss: 0.4259 - accuracy: 0.8010

2022-08-29 09:38:04.837040: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 240000000 exceeds 10% of free system memory.


782/782 [==============================] - 6s 7ms/step - loss: 0.4260 - accuracy: 0.8030 - val_loss: 0.4707 - val_accuracy: 0.7711
Epoch 6/10
 19/782 [..............................] - ETA: 4s - loss: 0.4092 - accuracy: 0.8092

2022-08-29 09:38:10.622754: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 240000000 exceeds 10% of free system memory.


782/782 [==============================] - 6s 7ms/step - loss: 0.4049 - accuracy: 0.8162 - val_loss: 0.4607 - val_accuracy: 0.7779
Epoch 7/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3894 - accuracy: 0.8242 - val_loss: 0.4539 - val_accuracy: 0.7825
Epoch 8/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3772 - accuracy: 0.8311 - val_loss: 0.4521 - val_accuracy: 0.7846
Epoch 9/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3678 - accuracy: 0.8372 - val_loss: 0.4490 - val_accuracy: 0.7863
Epoch 10/10
782/782 [==============================] - 5s 7ms/step - loss: 0.3607 - accuracy: 0.8416 - val_loss: 0.4478 - val_accuracy: 0.7883


In [13]:
score, acc = model_rnn.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

782/782 [==============================] - 1s 1ms/step - loss: 0.4478 - accuracy: 0.7883
Test score: 0.4477556347846985
Test accuracy: 0.7882800102233887


In [14]:
%tensorboard --logdir logs/fit

## Exercise

In this exercise, we will illustrate:
- Preparing the data to use sequences of length 80 rather than length 30.  Does it improve the performance?
- Trying different values of the "max_features".  Does this  improve the performance?
- Trying smaller and larger sizes of the RNN hidden dimension.  How does it affect the model performance?  How does it affect the run time?

In [15]:
max_features = 20000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [16]:
rnn_hidden_dim = 5
word_embedding_dim = 50
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [17]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

/home/damminhtien/anaconda3/envs/base-ml/lib/python3.9/site-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [18]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 11s 13ms/step - loss: 0.6141 - accuracy: 0.6517 - val_loss: 0.5329 - val_accuracy: 0.7245
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.4435 - accuracy: 0.7934 - val_loss: 0.4353 - val_accuracy: 0.7954
Epoch 3/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3699 - accuracy: 0.8379 - val_loss: 0.4037 - val_accuracy: 0.8168
Epoch 4/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3300 - accuracy: 0.8578 - val_loss: 0.3856 - val_accuracy: 0.8270
Epoch 5/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3053 - accuracy: 0.8717 - val_loss: 0.3773 - val_accuracy: 0.8325
Epoch 6/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2871 - accuracy: 0.8808 - val_loss: 0.3827 - val_accuracy: 0.8291
Epoch 7/10
782/782 [==============================] - 10s 13ms/step - loss: 0.2732 - accuracy: 0.8872 - val_loss: 0.3650 - val_accuracy:

In [19]:
max_features = 5000  # This is used in loading the data, picks the most common (max_features) words
maxlen = 80  # maximum length of a sequence - truncate after this

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

In [20]:
rnn_hidden_dim = 5
word_embedding_dim = 20
model_rnn = Sequential()
model_rnn.add(Embedding(max_features, word_embedding_dim))  #This layer takes each integer in the sequence
model_rnn.add(SimpleRNN(rnn_hidden_dim,
                    kernel_initializer=initializers.RandomNormal(stddev=0.001),
                    recurrent_initializer=initializers.Identity(gain=1.0),
                    activation='relu',
                    input_shape=x_train.shape[1:]))

model_rnn.add(Dense(1, activation='sigmoid'))

In [21]:
rmsprop = keras.optimizers.RMSprop(lr = .0001)

model_rnn.compile(loss='binary_crossentropy',
              optimizer=rmsprop,
              metrics=['accuracy'])

In [22]:
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 9s 10ms/step - loss: 0.6449 - accuracy: 0.6084 - val_loss: 0.5822 - val_accuracy: 0.6856
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4998 - accuracy: 0.7585 - val_loss: 0.4977 - val_accuracy: 0.7576
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.4321 - accuracy: 0.8008 - val_loss: 0.4371 - val_accuracy: 0.7984
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3888 - accuracy: 0.8291 - val_loss: 0.4163 - val_accuracy: 0.8109
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3609 - accuracy: 0.8444 - val_loss: 0.4398 - val_accuracy: 0.7959
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.3424 - accuracy: 0.8536 - val_loss: 0.3798 - val_accuracy: 0.8297
Epoch 7/10
782/782 [==============================] - 7s 10ms/step - loss: 0.3292 - accuracy: 0.8592 - val_loss: 0.3806 - val_accuracy: 0.8270

In [23]:
# Out of curiosity, run for 10 more epochs
model_rnn.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2977 - accuracy: 0.8744 - val_loss: 0.3768 - val_accuracy: 0.8304
Epoch 2/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2926 - accuracy: 0.8774 - val_loss: 0.3612 - val_accuracy: 0.8406
Epoch 3/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2885 - accuracy: 0.8772 - val_loss: 0.3625 - val_accuracy: 0.8404
Epoch 4/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2848 - accuracy: 0.8812 - val_loss: 0.3671 - val_accuracy: 0.8369
Epoch 5/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2817 - accuracy: 0.8814 - val_loss: 0.3574 - val_accuracy: 0.8434
Epoch 6/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2790 - accuracy: 0.8837 - val_loss: 0.3656 - val_accuracy: 0.8398
Epoch 7/10
782/782 [==============================] - 8s 10ms/step - loss: 0.2770 - accuracy: 0.8847 - val_loss: 0.3619 - val_accuracy: 0.8422

---
### Machine Learning Foundation (C) 2020 IBM Corporation